In [ ]:
r_de_load = requests.get(
    "https://api.energy-charts.info/public_power?country=de&start=2022-12-31T23:00Z&end=2024-12-31T22:45Z")
r_ch_load = requests.get(
    "https://api.energy-charts.info/public_power?country=ch&start=2022-12-31T23:00Z&end=2024-12-31T22:00Z")
r_dk_load = requests.get(
    "https://api.energy-charts.info/public_power?country=dk&start=2022-12-31T23:00Z&end=2024-12-31T22:00Z")
r_fr_load = requests.get(
    "https://api.energy-charts.info/public_power?country=fr&start=2022-12-31T23:00Z&end=2024-12-31T22:00Z")

r_de_prices = requests.get(
    "https://api.energy-charts.info/price?bzn=DE-LU&start=2022-12-31T23:00Z&end=2024-12-31T22:00Z")
r_ch_prices = requests.get("https://api.energy-charts.info/price?bzn=CH&start=2022-12-31T23:00Z&end=2024-12-31T22:00Z")
r_dk1_prices = requests.get(
    "https://api.energy-charts.info/price?bzn=DK1&start=2022-12-31T23:00Z&end=2024-12-31T22:00Z")
r_dk2_prices = requests.get(
    "https://api.energy-charts.info/price?bzn=DK2&start=2022-12-31T23:00Z&end=2024-12-31T22:00Z")
r_fr_prices = requests.get("https://api.energy-charts.info/price?bzn=FR&start=2022-12-31T23:00Z&end=2024-12-31T22:00Z")
r = [r_de_load, r_ch_load, r_dk_load, r_fr_load, r_de_prices, r_ch_prices, r_dk1_prices, r_dk2_prices, r_fr_prices]
for _ in r:
    print(_.ok)
timestamp = [ts.json()['unix_seconds'] for ts in r]
# give me all keys and all name values of the production data key
for i in range(len(r)):
    for key, value in r[i].json().items():
        if key == 'production_types':
            print(f"Key: {key}")
            val = [val['name'] for val in value]
            print(f"Value: {val}")
# construct a dataframe containing the data from the requests. attention: from ch, dk and fr load sets i only want to retrieve the entire load. from de load i only want lignite, hard coal, gas, wind onshore, wind offshore, solar and the overall load. the request r_de_load has to be resampled because its frequency is 15 min so only take the value for every full hour
df = pd.DataFrame()

for i in range(len(r)):
    for key, value in r[i].json().items():
        if key == 'production_types':
            for val in value:
                if val['name'] in ['Lignite', 'Hard Coal', 'Gas', 'Wind Onshore', 'Wind Offshore', 'Solar',
                                   'Total Load']:
                    df[val['name']] = pd.Series(val['data']).resample('H').mean()
        elif key == 'price':
            df['Price'] = pd.Series(value).resample('H').mean()
        elif key == 'total_load':
            df['Total Load'] = pd.Series(value).resample('H').mean()